In [2]:
%reset -f

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Mounted at /content/drive


In [ ]:
import ee

ee.Initialize(project='ee-andrewfullhart')

ic = ee.ImageCollection('NASA/NEX-GDDP')

study_area = ee.FeatureCollection('users/andrewfullhart/SW_Study_Area')

ndays_months = ee.List([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
months_index_global = ee.List.sequence(0, order_months.size().subtract(1))



inFILE = '/content/drive/My Drive/Colab Notebooks/GHCN_Historical_Annual_Mean_MEANP_Trends_Results.csv'

with open(inFILE) as f:
  lines = f.readlines()

points_ft_list = ee.List([])
for line in lines[1:]:
  row = line.strip('\n').split(',')
  lon = float(row[1])
  lat = float(row[2])
  ft = ee.Feature(None, {}).setGeometry(ee.Geometry.Point(lon, lat))
  points_ft_list = points_ft_list.add(ft)
points_fc = ee.FeatureCollection(points_ft_list)

#first_im = ic.first()
#points_fc = first_im.sample({'region':study_area, 'geometries':True})
#points_fc = ee.FeatureCollection(points_fc.toList(points_fc.size()).slice(0, 10))
points_fc_list = points_fc.toList(points_fc.size())



start_year = 2014
end_year = 2070
years_list_global = ee.List.sequence(start_year, end_year)
years_index_global = ee.List.sequence(0, years_list_global.size().subtract(1))

def pointlabel_fn(point):
  point = ee.Feature(point).geometry()
  a_strng = ee.Number(point.coordinates().get(0)).format('%.3f')
  b_strng = ee.String('_')
  c_strng = ee.Number(point.coordinates().get(1)).format('%.3f')
  point_strng = a_strng.cat(b_strng).cat(c_strng)
  point_strng = point_strng.replace('\\.', '_', 'g').slice(1)
  return point_strng


point_labels_global = points_fc.toList(points_fc.size()).map(pointlabel_fn);
point_index_global = ee.List.sequence(0, point_labels_global.size().subtract(1));

modelfilter = ee.Filter.Or(
              ee.Filter.eq('scenario', 'historical'),
              ee.Filter.eq('scenario', 'rcp45'))
ic = ic.filter(modelfilter)
ic = ic.filter(ee.Filter.eq('model', 'CCSM4')).select('pr')


global_years_list = ee.List.sequence(start_year, end_year)

modelfilter = ee.Filter.Or(
              ee.Filter.eq('scenario', 'historical'),
              ee.Filter.eq('scenario', 'rcp45'))

ic = ic.filter(modelfilter)

dict_list = []

for year in global_years_list.getInfo():

  start_year = year
  end_year = year
  years_list = ee.List.sequence(start_year, end_year)
  years_index = ee.List.sequence(0, years_list.size().subtract(1))

  def year_fn(year):
    year = ee.Number(year)
    start = ee.Date.fromYMD(ee.Number(year), 1, 1)
    end = ee.Date.fromYMD(ee.Number(year).add(30), 1, 1)
    year_ic = ic.filterDate(start, end)

    def month_fn(month):
      month = ee.Number(month)
      monthfilter = ee.Filter.calendarRange(month, month, 'month')
      month_ic = year_ic.filter(monthfilter)

      def point_fn(point):
        point = ee.Feature(point).geometry()
        data_list = month_ic.getRegion(point, 500).slice(1)

        def feature_fn(l):
          l = ee.List(l)
          date_str = ee.String(ee.String(l.get(0)).split('_').get(-1))
          year = ee.Number(date_str.slice(0, 4))
          month = ee.Number(date_str.slice(4, 6))
          prcp = ee.Number(l.get(4))
          prop_dict = {'precip':prcp}
          point_ft = ee.Feature(None, prop_dict)
          return point_ft

        data_fc = ee.FeatureCollection(data_list.map(feature_fn))
        nonzero_fc = data_fc.filter(ee.Filter.gt('precip', 0))
        mean = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.mean(), ['precip']).get('mean')).multiply(86400)
        mean = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), mean, 0))
        stdDev = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.stdDev(), ['precip']).get('stdDev')).multiply(86400)
        stdDev = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), stdDev, 0))
        skew = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.skew(), ['precip']).get('skew'))
        skew = ee.Number(ee.Algorithms.If(stdDev.eq(1), 0, skew))
        skew = ee.Number(ee.Algorithms.If(nonzero_fc.size().gt(0), skew, 0))
        ndays = ee.Number(nonzero_fc.size())
        number_ft = ee.Feature(None, {'mean':mean, 'sdev':stdDev, 'skew':skew, 'ndays':ndays})
        return number_ft

      means_fc = ee.FeatureCollection(points_fc.map(point_fn))
      return means_fc

    means_fc = ee.FeatureCollection(order_months.map(month_fn))
    return means_fc

  nested_fc = ee.FeatureCollection(years_list.map(year_fn))
  nested_fc_list = nested_fc.toList(years_list.size())

  def year_flatten_fn(year_i):
    year_i = ee.Number(year_i)
    year = years_list.get(year_i)
    month_fc_list = ee.FeatureCollection(nested_fc_list.get(year_i)).toList(order_months.size())

    def month_flatten_fn(month_i):
      month = ee.Number(month_i).add(1)
      subpoints_fc_list = ee.FeatureCollection(month_fc_list.get(month_i)).toList(points_fc.size())

      def point_flatten_fn(point_i):
        point_i = ee.Number(point_i)
        point_ft = ee.Feature(subpoints_fc_list.get(point_i))
        point_label = ee.String(point_labels_global.get(point_i))
        mean = point_ft.get('mean')
        sdev = point_ft.get('sdev')
        skew = point_ft.get('skew')
        ndays = point_ft.get('ndays')
        prop_dict = {'point':point_label, 'year':year, 'month':month, 'mean':mean, 'sdev':sdev, 'skew':skew, 'ndays':ndays}
        return ee.Feature(None, prop_dict)

      subflat_fc = ee.FeatureCollection(point_index_global.map(point_flatten_fn))
      return subflat_fc

    flat_fc = ee.FeatureCollection(months_index_global.map(month_flatten_fn)).flatten()
    return flat_fc


  out_fc = ee.FeatureCollection(years_index.map(year_flatten_fn)).flatten()


  taskA = ee.batch.Export.table.toDrive(collection=out_fc,
                              folder='GEE_Downloads',
                              description='NEX_Daily_Pars_30Yearly_MEAN_{}'.format(year),
                              selectors=['point', 'year', 'month', 'mean'])

  taskA.start()

  taskB = ee.batch.Export.table.toDrive(collection=out_fc,
                              folder='GEE_Downloads',
                              description='NEX_Daily_Pars_30Yearly_SDEV_{}'.format(year),
                              selectors=['point', 'year', 'month', 'sdev'])

  taskB.start()

  taskC = ee.batch.Export.table.toDrive(collection=out_fc,
                              folder='GEE_Downloads',
                              description='NEX_Daily_Pars_30Yearly_SKEW_{}'.format(year),
                              selectors=['point', 'year', 'month', 'skew'])

  taskC.start()

  taskD = ee.batch.Export.table.toDrive(collection=out_fc,
                                folder='GEE_Downloads',
                                description='NEX_Daily_Pars_30Yearly_NDAYS_{}'.format(year),
                                selectors=['point', 'year', 'month', 'ndays'])

  taskD.start()


In [39]:
#ASSUMES NO EXTRA FILES IN DIRECTORY
from collections import OrderedDict
import os

inDIR = '/content/drive/My Drive/GEE_Downloads'
outDIR = '/content/drive/My Drive/Colab Notebooks'


inFILES = os.listdir(inDIR)
stats = ['MEANP', 'SDEV', 'SKEW', 'NDAYS']
file_dict = {s:[] for s in stats}
for stat in stats:
  for f in inFILES:
    finfo = f.split('_')
    if finfo[4] == stat:
      file_dict[stat].append(f)

for stat in stats:
  file_dict[stat] = list(sorted(file_dict[stat]))

hdr_lines = []
for stat in stats:
  with open(os.path.join(inDIR, file_dict[stat][0])) as f:
    lines = f.readlines()
  hdr_line = lines[0]
  hdr_lines.append(hdr_line)

lines_dict = {}

for stat in stats:

  concat_lines = []

  for fi in file_dict[stat]:

    with open(os.path.join(inDIR, fi)) as f:

      lines = f.readlines()
      concat_lines.extend(lines[1:])

  stationIDs, years, months = [], [], []
  dummyStationID = concat_lines[0].split(',')[0]
  for line in concat_lines:
    row = line.strip('\n').split(',')
    stationIDs.append(row[0])
    if row[0] == dummyStationID:
      years.append(str(int(float(row[1]))))
      months.append(str(int(float(row[2]))))

  stationIDs = list(OrderedDict.fromkeys(stationIDs))

  stationData_Dict = {iD:[] for iD in stationIDs}
  for i, iD in enumerate(stationIDs):
    j = i
    done = False
    while done == False:
      row = concat_lines[j].strip('\n').split(',')
      data = row[3]
      if data == '':
        data = '0.'
      stationData_Dict[iD].append(data)
      j += len(stationIDs)
      if j > ((len(concat_lines)/len(stationIDs))*len(stationIDs))-1:
        done = True

  fileName = 'NEX_Daily_Pars_30Yearly_{}.csv'.format(stat)

  with open(os.path.join(outDIR, fileName), 'w') as fo:
    fo.write('year,month,')
    fo.write(','.join([iD for iD in stationIDs]) + '\n')
    for i in range(0, int(len(concat_lines)/len(stationIDs))):
      fo.write(years[i] + ',' + months[i] + ',')
      fo.write(','.join([stationData_Dict[iD][i] for iD in stationIDs]) + '\n')









['NEX_Daily_Pars_30Yearly_NDAYS_2014.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2015.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2014.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2014.csv', 'NEX_Daily_Pars_30Yearly_NDAYS_2015.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2015.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2014.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2015.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2016.csv', 'NEX_Daily_Pars_30Yearly_NDAYS_2016.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2016.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2016.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2017.csv', 'NEX_Daily_Pars_30Yearly_NDAYS_2017.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2017.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2017.csv', 'NEX_Daily_Pars_30Yearly_NDAYS_2018.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2018.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2018.csv', 'NEX_Daily_Pars_30Yearly_MEANP_2019.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2018.csv', 'NEX_Daily_Pars_30Yearly_SKEW_2019.csv', 'NEX_Daily_Pars_30Yearly_SDEV_2019.csv', 'NEX_Daily_Pars_30Yearly_NDAYS_2019.csv', 'NE